In [1]:
import pandas as pd
import csv, sqlite3

In [2]:
#install ipython-sql

!pip3 install ipython-sql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#connect to local

con = sqlite3.connect("chicago.db")
cur = con.cursor()

In [4]:
#load & connect sql

%load_ext sql

%sql sqlite:///chicago.db

In [5]:
#import from data to sql
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

566

In [6]:
#test connection & data
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///chicago.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


### Problem 1

##### Find the total number of crimes recorded in the CRIME table.

In [7]:
%sql select count(*) as total_crimes from chicago_crime_data;

 * sqlite:///chicago.db
Done.


total_crimes
533


### Problem 2

##### List community areas with per capita income less than 11000.

In [8]:
%sql select community_area_name from census_data where per_capita_income < 11000;

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [9]:
%sql select case_number from chicago_crime_data where description like '%MINOR%';

 * sqlite:///chicago.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4

##### List all kidnapping crimes involving a child?

In [10]:
%sql select case_number, primary_type, date, description from chicago_crime_data where primary_type = 'KIDNAPPING';

 * sqlite:///chicago.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DATE,DESCRIPTION
HN144152,KIDNAPPING,2007-01-26,CHILD ABDUCTION/STRANGER


### Problem 5

##### What kinds of crimes were recorded at schools?

In [11]:
%%sql 
select distinct(primary_type), 
    location_description 
from chicago_crime_data where location_description like '%SCHOOL%'

 * sqlite:///chicago.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


### Problem 6

##### List the average safety score for each type of school.

In [12]:
%%sql 
select "elementary, middle, or high school", 
    avg(safety_score) average_safety_score 
from chicago_public_schools_data 
group by "elementary, middle, or high school";

 * sqlite:///chicago.db
Done.


"Elementary, Middle, or High School",average_safety_score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7

##### List 5 community areas with highest % of households below poverty line 

In [13]:
%%sql 
select 
    community_area_name, 
    percent_households_below_poverty
from census_data 
order by percent_households_below_poverty desc 
limit 5 ;

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone?

In [14]:
%%sql
select 
    community_area_number, 
    count(community_area_number) as frequency
from chicago_crime_data 
group by community_area_number
order by count(community_area_number) desc
limit 1;

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NUMBER,frequency
25.0,43


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index

In [15]:
%%sql 
select community_area_name 
from census_data 
where hardship_index = (select max(hardship_index) from census_data);

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?

In [16]:
%%sql
select community_area_name
from   census_data
where  community_area_number = (select community_area_number
                                from   chicago_crime_data
                                group  by community_area_number
                                order  by count(community_area_number) desc
                                limit  1)
limit  1; 

 * sqlite:///chicago.db
Done.


COMMUNITY_AREA_NAME
Austin


## Author

<h4>Hanif Sudira</h4>